In [1]:
import math

import matplotlib as mpl
from matplotlib import cm
mpl.rcParams['text.usetex'] = True 
mpl.rcParams['text.latex.preamble'] = [r'\usepackage{libertine}', r'\usepackage{newtxmath}'] 
mpl.rc('font', family='serif')

import numpy as np
import pandas as pd
import seaborn as sns
import copy

import matplotlib.pyplot as plt
from matplotlib import rc

from cycler import cycler

from scipy.interpolate import make_interp_spline, BSpline

mpl.rcParams['ps.usedistiller'] = 'xpdf' 

plt.style.use('seaborn-notebook')

plt.rcParams['axes.titlesize'] = '25'
plt.rcParams['axes.labelsize'] = '25'
plt.rcParams['legend.fontsize'] = '15'
plt.rcParams['xtick.labelsize'] = '18'
plt.rcParams['ytick.labelsize'] = '18'

monochrome=(cycler('color', sns.color_palette("husl", 8))*2+(cycler('marker', ['v', 's', "o"])*7)[0:16])
plt.rc('axes', prop_cycle=monochrome)

pd.options.display.max_rows = 999
pd.options.display.max_columns = None
cmap = cm.get_cmap('RdBu')

husl4 = sns.color_palette("husl", 4)
husl5 = sns.color_palette("husl", 5)
husl6 = sns.color_palette("husl", 7)
greens5 = sns.color_palette("Greens_r", 5)
purples5 = sns.color_palette("Purples_r", 7)
blues5 = sns.color_palette("Blues_r", 5)
reds5 = sns.color_palette("Reds_r", 8)
oranges5 = sns.color_palette("Oranges_r", 5)
greys5 = sns.color_palette("Greys_r", 5)
winter5 = sns.color_palette("viridis_r", 5)

In [2]:
folder = "2020-04-29-16-49_StreamHiCSminers_" # Replace

In [3]:
summary = pd.read_csv("../experiments/" + folder + "/StreamHiCSminers.csv")

In [4]:
summary = summary[summary["subspaces"] != "Full"]

In [5]:
summary.head()

,refId,minerId,subspaces,minetime,rocauc,prauc,ap,p1,p2,p5,p10,p20,p30,r1,r2,r5,r10,r20,r30
0,activity,Hash,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,13174.5868,0.7243,NaN,0.3253,0.8378,0.6697,0.4676,0.3070,0.2234,0.1915,0.0836,0.1339,0.2337,0.3070,0.4467,0.5744
1,activity,LOF-2,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,27678.4547,0.7591,NaN,0.3603,0.7027,0.6360,0.5414,0.4081,0.2780,0.2181,0.0701,0.1272,0.2706,0.4081,0.5560,0.6544
2,activity,LOF-10,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,31621.8414,0.8385,NaN,0.5561,0.9685,0.8652,0.7203,0.5303,0.3463,0.2607,0.0966,0.1730,0.3600,0.5303,0.6926,0.7820
3,activity,LOF-50,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,51304.8965,0.8852,NaN,0.4738,0.7072,0.5461,0.5189,0.5083,0.3834,0.2917,0.0706,0.1092,0.2593,0.5083,0.7667,0.8751
4,activity,LOF-1,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,26166.2761,0.6630,NaN,0.1695,0.1802,0.1820,0.2131,0.2180,0.1946,0.1714,0.0180,0.0364,0.1065,0.2180,0.3892,0.5142


In [6]:
interesting = ["minetime", "rocauc", "ap", "p1", "p2", "p5", "r1", "r2", "r5"]

In [7]:
summary.loc[summary.groupby(["refId"])["rocauc"].idxmax()].groupby(["refId", "minerId"])[interesting].mean()*100

,,minetime,rocauc,ap,p1,p2,p5,r1,r2,r5
refId,minerId,,,,,,,,,
activity,LOF-50,5130489.65,88.52,47.38,70.72,54.61,51.89,7.06,10.92,25.93
example_10,LOF-5,709177.48,88.81,31.16,33.00,19.00,10.40,38.37,44.19,60.47
example_20,LOF-2,1599673.97,71.71,5.37,8.00,6.00,4.00,9.09,13.64,22.73
example_50,LOF-5,4111288.81,63.90,12.00,11.00,6.00,3.40,13.58,14.81,20.99
kdd99,LOF-2,1012275.97,82.09,15.98,1.20,0.60,0.24,0.17,0.17,0.17


In [8]:
summary.loc[summary.groupby(["refId"])["rocauc"].idxmax()]

,refId,minerId,subspaces,minetime,rocauc,prauc,ap,p1,p2,p5,p10,p20,p30,r1,r2,r5,r10,r20,r30
3,activity,LOF-50,/home/edouardfouche/data/subspaces-StreamHiCS/acti...,51304.8965,0.8852,NaN,0.4738,0.7072,0.5461,0.5189,0.5083,0.3834,0.2917,0.0706,0.1092,0.2593,0.5083,0.7667,0.8751
34,example_10,LOF-5,/home/edouardfouche/data/subspaces-StreamHiCS/exam...,7091.7748,0.8881,0.3116,0.3116,0.3300,0.1900,0.1040,0.0620,0.0355,0.0253,0.3837,0.4419,0.6047,0.7209,0.8256,0.8837
50,example_20,LOF-2,/home/edouardfouche/data/subspaces-StreamHiCS/exam...,15996.7397,0.7171,0.0537,0.0537,0.0800,0.0600,0.0400,0.0330,0.0245,0.0197,0.0909,0.1364,0.2273,0.3750,0.5568,0.6705
71,example_50,LOF-5,/home/edouardfouche/data/subspaces-StreamHiCS/exam...,41112.8881,0.6390,0.1200,0.1200,0.1100,0.0600,0.0340,0.0220,0.0180,0.0140,0.1358,0.1481,0.2099,0.2716,0.4444,0.5185
21,kdd99,LOF-2,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,10122.7597,0.8209,0.0738,0.1598,0.0120,0.0060,0.0024,0.0200,0.1178,0.2372,0.0017,0.0017,0.0017,0.0281,0.3311,1.0000


In [9]:
summary[summary["refId"] == "kdd99"]

,refId,minerId,subspaces,minetime,rocauc,prauc,ap,p1,p2,p5,p10,p20,p30,r1,r2,r5,r10,r20,r30
16,kdd99,Hash,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,2669.6901,0.4729,0.0670,0.0638,0.000,0.000,0.0008,0.0008,0.0198,0.0664,0.0000,0.0000,0.0006,0.0011,0.0556,0.2799
17,kdd99,LOF-10,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,18777.8565,0.7845,0.0757,0.1398,0.016,0.008,0.0032,0.0016,0.0828,0.2372,0.0022,0.0022,0.0022,0.0022,0.2327,1.0000
18,kdd99,LOF-50,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,19822.2603,0.7066,0.0755,0.1086,0.000,0.000,0.0024,0.0152,0.0076,0.0800,0.0000,0.0000,0.0017,0.0214,0.0214,0.3373
19,kdd99,LOF-1,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,18548.2760,0.8186,0.0711,0.1570,0.004,0.002,0.0008,0.0456,0.1174,0.2372,0.0006,0.0006,0.0006,0.0641,0.3300,1.0000
20,kdd99,LOF-100,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,21514.8573,0.5711,0.0792,0.0789,0.000,0.000,0.0008,0.0004,0.0006,0.0004,0.0000,0.0000,0.0006,0.0006,0.0017,0.0017
21,kdd99,LOF-2,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,10122.7597,0.8209,0.0738,0.1598,0.012,0.006,0.0024,0.0200,0.1178,0.2372,0.0017,0.0017,0.0017,0.0281,0.3311,1.0000
22,kdd99,LOF-20,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,13042.6214,0.7464,0.0757,0.1231,0.008,0.006,0.0024,0.0012,0.0170,0.1772,0.0011,0.0017,0.0017,0.0017,0.0478,0.7470
23,kdd99,LOF-5,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,9574.4588,0.8019,0.0748,0.1489,0.016,0.008,0.0032,0.0016,0.1132,0.2372,0.0022,0.0022,0.0022,0.0022,0.3182,1.0000


In [10]:
summary[summary["refId"] == "kdd99"].groupby("minerId")[interesting].mean() * 100

,minetime,rocauc,ap,p1,p2,p5,r1,r2,r5
minerId,,,,,,,,,
Hash,266969.01,47.29,6.38,0.0,0.0,0.08,0.00,0.00,0.06
LOF-1,1854827.60,81.86,15.70,0.4,0.2,0.08,0.06,0.06,0.06
LOF-10,1877785.65,78.45,13.98,1.6,0.8,0.32,0.22,0.22,0.22
LOF-100,2151485.73,57.11,7.89,0.0,0.0,0.08,0.00,0.00,0.06
LOF-2,1012275.97,82.09,15.98,1.2,0.6,0.24,0.17,0.17,0.17
LOF-20,1304262.14,74.64,12.31,0.8,0.6,0.24,0.11,0.17,0.17
LOF-5,957445.88,80.19,14.89,1.6,0.8,0.32,0.22,0.22,0.22
LOF-50,1982226.03,70.66,10.86,0.0,0.0,0.24,0.00,0.00,0.17


In [11]:
summary[summary["refId"] == "kdd99"][["refId", "minerId", "subspaces"] + interesting]

,refId,minerId,subspaces,minetime,rocauc,ap,p1,p2,p5,r1,r2,r5
16,kdd99,Hash,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,2669.6901,0.4729,0.0638,0.000,0.000,0.0008,0.0000,0.0000,0.0006
17,kdd99,LOF-10,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,18777.8565,0.7845,0.1398,0.016,0.008,0.0032,0.0022,0.0022,0.0022
18,kdd99,LOF-50,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,19822.2603,0.7066,0.1086,0.000,0.000,0.0024,0.0000,0.0000,0.0017
19,kdd99,LOF-1,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,18548.2760,0.8186,0.1570,0.004,0.002,0.0008,0.0006,0.0006,0.0006
20,kdd99,LOF-100,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,21514.8573,0.5711,0.0789,0.000,0.000,0.0008,0.0000,0.0000,0.0006
21,kdd99,LOF-2,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,10122.7597,0.8209,0.1598,0.012,0.006,0.0024,0.0017,0.0017,0.0017
22,kdd99,LOF-20,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,13042.6214,0.7464,0.1231,0.008,0.006,0.0024,0.0011,0.0017,0.0017
23,kdd99,LOF-5,/home/edouardfouche/data/subspaces-StreamHiCS/kddc...,9574.4588,0.8019,0.1489,0.016,0.008,0.0032,0.0022,0.0022,0.0022
